In [1]:
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
Data = pd.read_csv("Data/train.csv")

C:\Users\Steek\AppData\Local\Temp\ipykernel_14712\3929139985.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  Data = pd.read_csv("Data/train.csv")


In [3]:
holiday_map = {
    "0": 0,
    "a": 1,
    "b": 2,
    "c": 3
}
Data["StateHoliday"] = Data["StateHoliday"].map(holiday_map).fillna(0).astype(int)
Data["Date"] = pd.to_datetime(Data["Date"])

In [4]:
Data = Data.sort_values("Date")
Data

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
1017208,1115,2,2013-01-01,0,0,0,0,1,1
1016473,379,2,2013-01-01,0,0,0,0,1,1
1016472,378,2,2013-01-01,0,0,0,0,1,1
1016471,377,2,2013-01-01,0,0,0,0,1,1
1016470,376,2,2013-01-01,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...
745,746,5,2015-07-31,9082,638,1,1,0,1
746,747,5,2015-07-31,10708,826,1,1,0,1
747,748,5,2015-07-31,7481,578,1,1,0,1
741,742,5,2015-07-31,10460,1016,1,1,0,1


In [5]:
def RMSPE(y_true, y_pred):
    mask = y_true!=0
    y_true=y_true[mask]
    y_pred = y_pred[mask]
    PE = (y_true-y_pred)/y_true
    return np.sqrt(np.mean(PE**2))
Feet = Data.sort_values("Date")
def RunModel(model,Target,Feet):
    TSS_cv = TimeSeriesSplit(n_splits=5) #Some leak here since it probably has some stores on same day in two sections but probably irrelevant. 
    for train_idx,test_idx in TSS_cv.split(Feet):
        #Splits DAta
        Feet_train, Feet_test = Feet.iloc[train_idx],Feet.iloc[test_idx]
        Target_train,Target_test = Target.iloc[train_idx],Target.iloc[test_idx]
        #Train model
        model.fit(Feet_train,Target_train)
        #Predictions
        train_pred = model.predict(Feet_train)
        train_error = RMSPE(Target_train,train_pred)
        preds = model.predict(Feet_test)
        error = RMSPE(Target_test, preds)
        print(f"Training error was: {100*train_error:.2f} % and target error {100*error:.2f} %")
        


In [42]:
#Simple first pass with a 🎄🎄🎄 or a BOOST
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb

Feature_list = ["Store","DayOfWeek","Promo","StateHoliday","SchoolHoliday"]

small_Data = Data[Data["Date"]<"2015-06-01"]
Target = small_Data["Sales"]
Feet = small_Data[Feature_list+["Open"]] #🦶🏻 Maybe toes are better, then feet is the dataset? 

#Define models
XGB_model = xgb.XGBRegressor(n_estimators=100,learning_rate=0.1,max_depth=10,n_jobs=-1,random_state=404)
RF_model = RandomForestRegressor(n_estimators=100,max_depth=15, n_jobs=-1,random_state=404)

#RunModel(RF_model,Target,Feet)
print("XGB")
RunModel(XGB_model,Target,Feet)

XGB
Training error was: 51.05 % and target error 48.99 %
Training error was: 48.79 % and target error 43.16 %
Training error was: 49.06 % and target error 42.91 %
Training error was: 47.67 % and target error 57.05 %
Training error was: 50.34 % and target error 41.56 %


In [43]:
#50% error sure is bad. Lets try some stuff
#Adding the store information
Store_info = pd.read_csv("Data/store.csv")
Data_store = pd.merge(Data,Store_info,on="Store",how="left")
Data_store['CompetitionDistance'] = Data_store['CompetitionDistance'].fillna(100000)
Data_store['CompetitionOpenSinceMonth'] = Data_store['CompetitionOpenSinceMonth'].fillna(0)
Data_store['CompetitionOpenSinceYear'] = Data_store['CompetitionOpenSinceYear'].fillna(0)
#Remove closed days and add month/day.
small_Data = Data_store[Data_store["Date"]<"2014-06-01"]
small_Data = small_Data[small_Data["Open"]!=0].drop(columns="Open")
small_Data["Month"] = small_Data["Date"].dt.month
small_Data["Day"] = small_Data["Date"].dt.day

Feature_list = ["Store","DayOfWeek","Promo","StateHoliday","SchoolHoliday","Day","Month"]
Store_features = ["CompetitionDistance","CompetitionOpenSinceYear","Promo2"]

Target = small_Data["Sales"]
Feet = small_Data[Feature_list + Store_features]

RunModel(XGB_model,Target,Feet)

Training error was: 20.31 % and target error 35.33 %
Training error was: 21.37 % and target error 25.53 %
Training error was: 22.02 % and target error 18.72 %
Training error was: 22.80 % and target error 26.99 %
Training error was: 22.25 % and target error 21.26 %


In [44]:
#That helped a bit, but we can probably do better. 
SD = small_Data.copy()

#Does same as one-hot but trains faster so eh, not that important. 
SD["StoreType"] = SD["StoreType"].map(holiday_map).fillna(0).astype(int)

month_map = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
    7: 'Jul', 8: 'Aug', 9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}

def is_promo2_active(DFrow):
    if pd.isna(DFrow["PromoInterval"]):
        return 0
    Month = month_map[DFrow["Date"].month]

    if Month in DFrow["PromoInterval"]:
        return 1
    return 0

SD["IsPromo2Month"] = SD.apply(is_promo2_active,axis=1)



In [45]:
Store_features = ["CompetitionDistance","CompetitionOpenSinceYear","Promo2","IsPromo2Month"]

Target = SD["Sales"]
Feet = SD[Feature_list + Store_features]
#RunModel(RF_model,Target,Feet)
print("XGB")
RunModel(XGB_model,Target,Feet)

XGB
Training error was: 20.75 % and target error 35.63 %
Training error was: 21.55 % and target error 26.04 %
Training error was: 22.08 % and target error 18.88 %
Training error was: 22.34 % and target error 26.72 %
Training error was: 22.10 % and target error 20.85 %


In [46]:
#Try feature encoding where we tell it the mean sales of the store beforehand (but are careful to only use training info!)
def RunModelAdvanced(model,Data,features):
    TSS_cv = TimeSeriesSplit(n_splits=5)
    for train_idx,test_idx in TSS_cv.split(Data):
        #Splits DAta
        Data_train = Data.iloc[train_idx].copy()
        Data_test = Data.iloc[test_idx].copy()
        #Target_ecoding
        store_mean_sales = Data_train.groupby('Store')['Sales'].mean() #Adds column of mean store sales
        Data_train["Store_Mean_Sales"] = Data_train["Store"].map(store_mean_sales)
        Data_test["Store_Mean_Sales"] = Data_test["Store"].map(store_mean_sales)
        #Splits data into features and target
        Feet_train, Target_train = Data_train[features+["Store_Mean_Sales"]],Data_train["Sales"]
        Feet_test, Target_test = Data_test[features+["Store_Mean_Sales"]],Data_test["Sales"]
        #Train model
        model.fit(Feet_train,np.log1p(Target_train)) #Objective is log based to help MSE
        #Predictions
        train_pred = np.expm1(model.predict(Feet_train))
        train_error = RMSPE(Target_train,train_pred)
        preds = np.expm1(model.predict(Feet_test))
        error = RMSPE(Target_test, preds)
        print(f"Training error was: {100*train_error:.2f} % and target error {100*error:.2f} %")

#RunModelAdvanced(RF_model,SD,Feature_list+Store_features)
print("XGB")
RunModelAdvanced(XGB_model,SD,Feature_list+Store_features)

XGB
Training error was: 11.86 % and target error 34.36 %
Training error was: 11.51 % and target error 24.97 %
Training error was: 12.02 % and target error 15.57 %
Training error was: 12.58 % and target error 23.72 %
Training error was: 13.36 % and target error 15.56 %


In [67]:
#Final attempt.... Pre-cluster the stores into 10 groups to nudge a bit, idk.A bit lazy so its variance based clustering hehe, the mean is in the other row.  
from sklearn.cluster import KMeans
def RunModelAdvancedCluster(model,Data,features):
    TSS_cv = TimeSeriesSplit(n_splits=5)
    for train_idx,test_idx in TSS_cv.split(Data):
        #Splits DAta
        Data_train = Data.iloc[train_idx].copy()
        Data_test = Data.iloc[test_idx].copy()
        #Target_ecoding
        store_mean_sales = Data_train.groupby('Store')['Sales'].mean() #Adds column of mean store sales
        Data_train["Store_Mean_Sales"] = Data_train["Store"].map(store_mean_sales)
        Data_test["Store_Mean_Sales"] = Data_test["Store"].map(store_mean_sales)
        #Clustering
        store_profiles = Data_train.groupby('Store')[['Sales', 'Customers']].var()
        km = KMeans(n_clusters=10, n_init=10, random_state=42)
        store_profiles['Cluster'] = km.fit_predict(store_profiles)
        Data_train['Store_Group'] = Data_train['Store'].map(store_profiles['Cluster'])
        Data_test['Store_Group'] = Data_test['Store'].map(store_profiles['Cluster']).fillna(-1)
        #Splits data into features and target
        Feet_train, Target_train = Data_train[features+["Store_Mean_Sales","Store_Group"]],Data_train["Sales"]
        Feet_test, Target_test = Data_test[features+["Store_Mean_Sales","Store_Group"]],Data_test["Sales"]
        #Train model
        model.fit(Feet_train,np.log1p(Target_train)) #Objective is log based to help MSE
        #Predictions
        train_pred = np.expm1(model.predict(Feet_train))
        train_error = RMSPE(Target_train,train_pred)
        preds = np.expm1(model.predict(Feet_test))
        error = RMSPE(Target_test, preds)
        print(f"Training error was: {100*train_error:.2f} % and target error {100*error:.2f} %")

#RunModelAdvanced(RF_model,SD,Feature_list+Store_features)
print("XGB")
RunModelAdvancedCluster(XGB_model,SD,Feature_list+Store_features)

XGB
Training error was: 10.70 % and target error 33.10 %
Training error was: 10.73 % and target error 24.95 %
Training error was: 11.06 % and target error 15.20 %
Training error was: 11.57 % and target error 23.43 %
Training error was: 12.84 % and target error 15.36 %


In [59]:
#Less is more, or so they say, does all the fluff matter?
AllFeat = ["Store","DayOfWeek","Promo","StateHoliday","SchoolHoliday"]+["CompetitionDistance","CompetitionOpenSinceYear","Promo2","IsPromo2Month"]
RunModelAdvancedCluster(XGB_model,SD,["Promo","DayOfWeek","SchoolHoliday","CompetitionDistance"])

Training error was: 15.21 % and target error 32.34 %
Training error was: 21.07 % and target error 25.16 %
Training error was: 20.91 % and target error 16.15 %
Training error was: 20.32 % and target error 22.46 %
Training error was: 21.26 % and target error 15.75 %


In [65]:
#Try lgb
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor(n_estimators=500,learning_rate=0.1,num_leaves=60,n_jobs=-1,verbosity=-1)
RunModelAdvanced(lgb_model,SD,Feature_list+Store_features)

Training error was: 12.92 % and target error 33.66 %
Training error was: 14.91 % and target error 18.53 %
Training error was: 15.37 % and target error 15.27 %
Training error was: 15.50 % and target error 24.55 %
Training error was: 16.32 % and target error 15.96 %


In [68]:
SD

,Store,DayOfWeek,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Day,IsPromo2Month
27,353,2,2013-01-01,3139,820,0,1,1,2,b,900.0,0.0,0.0,1,14.0,2013.0,"Feb,May,Aug,Nov",1,1,0
115,335,2,2013-01-01,2401,482,0,1,1,2,a,90.0,0.0,0.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",1,1,1
147,512,2,2013-01-01,2646,625,0,1,1,2,b,590.0,0.0,0.0,1,5.0,2013.0,"Mar,Jun,Sept,Dec",1,1,0
162,494,2,2013-01-01,3113,527,0,1,1,2,a,1260.0,6.0,2011.0,0,NaN,NaN,NaN,1,1,0
199,530,2,2013-01-01,2907,532,0,1,1,1,c,18160.0,0.0,0.0,0,NaN,NaN,NaN,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575334,743,6,2014-05-31,1821,236,0,0,0,1,a,6710.0,11.0,2003.0,1,14.0,2012.0,"Jan,Apr,Jul,Oct",5,31,0
575335,744,6,2014-05-31,7589,829,0,0,0,1,a,1370.0,12.0,2011.0,1,40.0,2014.0,"Jan,Apr,Jul,Oct",5,31,0
575336,745,6,2014-05-31,3271,345,0,0,0,1,a,17650.0,11.0,2013.0,1,37.0,2009.0,"Jan,Apr,Jul,Oct",5,31,0
575337,746,6,2014-05-31,6447,562,0,0,0,0,c,4330.0,2.0,2011.0,1,35.0,2011.0,"Mar,Jun,Sept,Dec",5,31,0
